## Création d'une image

Le format d'une image est défini par un [standard ouvert]( https://github.com/opencontainers/image-spec/blob/main/spec.md). En plus de différentes métadonnées comme la command par défaut, une image est composée d'un ensemble de couches qui indiquent les changements effectués à partir de la couche précédente. Cela permet de partager efficacement des bases communes entre images mais demande quelques précautions.

### Manuellement

La commande `history` montre les couches qui composent une image.

In [53]:
docker history ubuntu:jammy

IMAGE          CREATED      CREATED BY                                      SIZE      COMMENT
730eeb702b69   8 days ago   /bin/sh -c #(nop)  CMD ["/bin/bash"]            0B        
<missing>      8 days ago   /bin/sh -c #(nop) ADD file:cf91de9ab30abab11…   69.2MB    
<missing>      8 days ago   /bin/sh -c #(nop)  LABEL org.opencontainers.…   0B        
<missing>      8 days ago   /bin/sh -c #(nop)  LABEL org.opencontainers.…   0B        
<missing>      8 days ago   /bin/sh -c #(nop)  ARG LAUNCHPAD_BUILD_ARCH     0B        
<missing>      8 days ago   /bin/sh -c #(nop)  ARG RELEASE                  0B        


Pour créer une image, il est possible d'exécuter un conteneur, de faire de faire des modifications manuelles puis de les valider. ce qui provoque la création d'une nouvelle couche dans le système de fichier et donc d'une nouvelle image. Cette approche est déconseillée car difficile à maintenir. 

In [60]:
docker run --name my-ubuntu-container --interactive ubuntu:jammy bash -  <<EOF
apt-get update &> /dev/null
apt-get install --yes --no-install-recommends &> /dev/null \
    git 
apt-get clean autoclean &> /dev/null
apt-get autoremove --yes &> /dev/null
rm -rf /var/lib/apt/lists/*
EOF

In [61]:
docker commit my-ubuntu-container mygit:latest

sha256:97b65dfa4d8ea43427008d42157ef55ae8202beb7058d90988f3301454fdf4b6


In [62]:
docker run --rm mygit git --version

git version 2.34.1


Nous pouvons voir que l'ID de l'avant-dernière couche est celui de la dernière de l'image d'ubuntu et qu'une nouvelle couche qui contient les modifications faite par les commandes précédentes a été ajoutée.

In [65]:
docker history mygit

IMAGE          CREATED              CREATED BY                                      SIZE      COMMENT
97b65dfa4d8e   About a minute ago   bash -                                          64MB      
730eeb702b69   8 days ago           /bin/sh -c #(nop)  CMD ["/bin/bash"]            0B        
<missing>      8 days ago           /bin/sh -c #(nop) ADD file:cf91de9ab30abab11…   69.2MB    
<missing>      8 days ago           /bin/sh -c #(nop)  LABEL org.opencontainers.…   0B        
<missing>      8 days ago           /bin/sh -c #(nop)  LABEL org.opencontainers.…   0B        
<missing>      8 days ago           /bin/sh -c #(nop)  ARG LAUNCHPAD_BUILD_ARCH     0B        
<missing>      8 days ago           /bin/sh -c #(nop)  ARG RELEASE                  0B        


In [66]:
docker container rm my-ubuntu-container
docker image rm mygit

my-ubuntu-container
Untagged: mygit:latest
Deleted: sha256:97b65dfa4d8ea43427008d42157ef55ae8202beb7058d90988f3301454fdf4b6
Deleted: sha256:c0821dc1dcbf52ae76f896bc2487dbead9cae93e77a6d1d71885d0c9564b97ca


### Dockerfile

L'approche recommandée est de créer un fichier `Dockerfile` (cf. https://docs.docker.com/engine/reference/builder/) pour automatiser la création d'une image. Ce fichier peut être édité lors des mises à jour et versionné avec git pour gérer un historique de l'infrastructure ainsi que des versions parallèles.

#### Structure de base

Le fichier `Dockerfile` suivant il les concepts de base. Il s'appuie sur l'image `python:3.11-slim-bullseye` (`FROM`), ajoute les fichiers de l'application depuis le contexte (`COPY`), exécute les commandes nécessaire à la construction de l'image (`RUN`) et défini la commande par défaut de l'image (`CMD`).

```{literalinclude} sample-python/helloworld/Dockerfile
:language: dockerfile
:caption: sample-python/helloworld/Dockerfile
:name: sample-python/helloworld/Dockerfile
```

La commande suivante construit une image tagguée à partir du Dockerfile contenu dans le répertoire courant (indiqué par le `.` à la fin).
C'est ce dernier paramètre qui est le contexte. Les options `--build-arg` permettre de fixer les `ARG` du Dockerfile.

In [167]:
( cd sample-python/helloworld && \
 docker image build \
     --quiet \
     --build-arg BUILD_DATE=$(date -u +'%Y-%m-%dT%H:%M:%SZ')\
     --tag brunoe/helloworld_python:0.1 \
     . 
)

sha256:b2c58167fa9fc542c6d74d6659465539328e8351400f049a3b7b4e1a542bc40b


d'autres tags peuvent être appliqués pour facilier l'usage de l'image, généralement des versions plus génériques (1.2.3 et 1.2 et 1) et/ou des informations sur l'image de base avec des variantes (1.2.3-ubuntu, 1.2.3-debian, ...) et/ou des variantes de constructions de l'image elle-même et toujours `latest`.

In [ ]:
docker tag brunoe/helloworld_python:0.1 brunoe/helloworld_python:0
docker tag brunoe/helloworld_python:0.1 brunoe/helloworld_python:latest

Cette image peut alors être utilisée localement.

In [142]:
docker run --rm brunoe/helloworld_python:0.1

In [143]:
docker run --env NAME="Pierre" --rm brunoe/helloworld_python:0.1

Il est intéressant de connaitre la taille d'une image et le détail des couches qui la compose. 

In [147]:
docker image ls|grep brunoe/helloworld_python|tr -s ' '

brunoe/helloworld_python 0 dee32cbd7470 7 minutes ago 290MB
brunoe/helloworld_python 0.1 dee32cbd7470 7 minutes ago 290MB
brunoe/helloworld_python latest dee32cbd7470 7 minutes ago 290MB


In [148]:
docker history brunoe/helloworld_python:0.1

IMAGE          CREATED         CREATED BY                                      SIZE      COMMENT
dee32cbd7470   7 minutes ago   CMD ["/bin/sh" "-c" "python ${WORKDIR}/hello…   0B        buildkit.dockerfile.v0
<missing>      7 minutes ago   COPY hello.py ./ # buildkit                     254B      buildkit.dockerfile.v0
<missing>      7 minutes ago   RUN |1 BUILD_DATE=2023-03-09T16:49:26Z /bin/…   169MB     buildkit.dockerfile.v0
<missing>      2 hours ago     COPY requirements.txt ./ # buildkit             13B       buildkit.dockerfile.v0
<missing>      2 hours ago     WORKDIR /                                       0B        buildkit.dockerfile.v0
<missing>      2 hours ago     ENV NAME=John Doe                               0B        buildkit.dockerfile.v0
<missing>      2 hours ago     LABEL org.label-schema.build-date=2023-03-09…   0B        buildkit.dockerfile.v0
<missing>      2 hours ago     LABEL maintainer=emmanuel.bruno@univ-tln.fr     0B        buildkit.dockerfile.v0
<missin

### Partage de l'image

Une image peut ensuite être partagée sur un registry (dockerhub ou un autre qui peut être public ou privé). Pour cela le docker client doit s'authentifier (`docker login`) puis pousser les différents tags de l'image (`docker push <image_name>`). Attention, le nom de l'image doit généralement préciser le compte sur le repository.

```bash
docker push brunoe/helloworld_python:0.1
docker push brunoe/helloworld_python:latest
```

### Entrypoint

La directive `ENTRYPOINT` prend en paramètre une commande permet que l'image docker se comporte comme cette commande en acceptant directement les paramètres lors d'un `run`. La commande exécutée est la concaténation de `ENTRYPOINT` et des paramètre de `run` ou de `CMD`. `CMD` correspond donc aux options par défaut que l'on souhaite si aucun paramètre n'est donné. 

L'exemple suivant montre comment construire une commande qui crée une base de donnée sqlite dans repertoire courant.

In [297]:
docker build --quiet --tag sqlite - <<EOF
FROM alpine
VOLUME /workdir
WORKDIR /workdir
RUN apk --no-cache add sqlite
ENTRYPOINT ["sqlite3","-box","local.db"]
CMD ["--version"]
EOF

sha256:d05825d9126d36c9e034fb4122221b7bbe3fae3dd0ac796e76e8eeeea754687c


In [290]:
docker run --rm sqlite

3.40.1 2022-12-28 14:03:47 df5c253c0b3dd24916e4ec7cf77d3db5294cc9fd45ae7b9c5e82ad8197f38a24


In [291]:
alias sqlite='docker run -it -v ${PWD}:/workdir --rm sqlite'
sqlite "DROP TABLE IF EXISTS data;"
sqlite "CREATE TABLE IF NOT EXISTS data (key TEXT PRIMARY KEY,value TEXT NOT NULL);"
sqlite "INSERT INTO data values('A','10');INSERT INTO data values('B','20');"
sqlite "SELECT * FROM data;"

┌─────┬───────┐
│ key │ value │
├─────┼───────┤
│ A   │ 10    │
│ B   │ 20    │
└─────┴───────┘


### Multistage

Un Dockerfile dit `multistage` utilise plusieurs `FROM`. Chacun définit une image mais seule la dernière sera le résultat final. Les images précédentes peuvent être utilisées pour produire des fichiers qui seront copiés dans une étape (stage) suivant. 

Pour cela, chaque `FROM` peut ête nommé avec `AS` pour qu'une étape suivante puisse accéder à son système de fichier avec `COPY --from=<stage_name> <source> <destination>`. 

L'utilisation la plus classique est d'avoir deux stages : (1) compilation et (2) exécution. La première étape contient l'environnement de développement (gcc pour le C, JDK+Maven pour Java, ...) et les sources,  elle produit l'exécutable (un binaire pour le C, du bytecode ou un jar pour Java, ...). La seconde étape ne contient que l'environnement d'exécution (éventuelles librairies dynamiques pour le C, JRE pour Java, ...) et l'exécutable.

```{literalinclude} sample-c/helloworld/Dockerfile
:language: dockerfile
:caption: sample-c/helloworld/Dockerfile
:name: sample-c/helloworld/Dockerfile
```

In [ ]:
( cd sample-c/helloworld/ && \
 docker image build \
     --quiet \
     --tag brunoe/helloworld_c:0.1 \
     . )

In [ ]:
docker run brunoe/helloworld_c:0.1

#### Volumes et réseaux

EXPOSE et VOLUME